# 양방향  LSTM +  CRF 이용 개체명 인식

##  CRF layer 사용을 위한 환경구성

- anaconda 가상 환경 구성( python==3.7.0 )   
  conda create -n tf113 anaconda python==3.7.0
   
      
- anaconda 가상 환경 활성화   
  conda activate tf113
   
      
- 활성화된 가상환경에서 tensorflow( 1.13.1 ), keras( 2.2.4 ), keras_contrib 설치   
   
(tf113)>conda install tensorflow==1.13.0   

(tf113)>conda install keras==2.2.4   

(tf113)>pip install git+https://www.github.com/keras-team/keras-contrib.git  -> 필요시 pip upgrade

### 개체명 인식을 위한 데이터 이해 및 전처리 

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

ValueError: source code string cannot contain null bytes

In [ ]:
data = pd.read.csv( 'ner_dataset.csv', encoding = 'latin1' )

In [ ]:
data[ :5 ]

In [ ]:
print( '데이터프레임 행의 개수 : {}'.format( len( data ) ) )

In [ ]:
print( '데이터에 null 값이 있는지 유무 : {}'.format( str( data.isnull().values.any() ) ) )

In [ ]:
print( '어떤 열에 null 값이 있는지 출력' )
print( '=================================' )
data.isnull().sum()

In [ ]:
print( 'sentence # 열의 중복을 제거한 값의 개수 : {}'.format( data[ 'Sentence #' ].nunique() ) )
print( 'Word 열의 중복을 제거한 값의 개수 : {}'.format( data.Word.nunique() ) )
print( 'Tag 열의 중복을 제거한 값의 개수 : {}'.format( data.Tag.nunique() ) )

In [ ]:
print( 'Tag 열의 각각의 값의 개수 카운트' )
print( '================================' )
print( data.groupby( 'Tag' ).size().reset_index( name = 'count' ) )

In [ ]:
data = data.fillna( method = "ffill" )

In [ ]:
print( data.tail() )

In [ ]:
print( '데이터에 Null 값이 있는지 유무 : ' + str( data.isnull().values.any() ) )

In [ ]:
data[ 'Word' ] = data[ 'Word' ].str.lower()
print( 'Word 열의 중복을 제거한 값의 개수 : {}'.format( data.Word.nunique() ) )

In [ ]:
print( data[ :5 ] )

In [ ]:
func = lambda temp : [ ( w, t ) for w, t in zip( temp[ "Word" ].values.tolist(), temp[ "Tag" ].values.tolist() ) ]
tagged_sentences = [ t for t in data.groupby( "Sentence #" ).apply( func ) ]
print( "전체 샘플 개수: {}".format( len( tagged_sentences ) ) )

In [ ]:
print( tagged_sentences[ 0 ] )

In [ ]:
sentences, ner_tags = [], [] 
for tagged_sentence in tagged_sentences: # 47,959개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip( *tagged_sentence ) # 각 샘플에서 단어들은 sentence에 개체명 태깅 정보들은 tag_info에 저장.
    sentences.append( list( sentence ) ) # 각 샘플에서 단어 정보만 저장한다.
    ner_tags.append( list( tag_info ) ) # 각 샘플에서 개체명 태깅 정보만 저장한다.

In [ ]:
print( sentences[ 0 ] )
print( ner_tags[ 0 ] )

In [ ]:
print( sentences[ 98 ] )
print( ner_tags[ 98 ] )

In [ ]:
print( '샘플의 최대 길이 : %d' % max( len( l ) for l in sentences ) )
print( '샘플의 평균 길이 : %f' % ( sum( map( len, sentences ) ) / len( sentences ) ) )
plt.hist( [ len( s ) for s in sentences] , bins = 50 )
plt.xlabel( 'length of samples' )
plt.ylabel( 'number of samples' )
plt.show()

### Tokenization

In [ ]:
src_tokenizer = Tokenizer( oov_token = 'OOV' ) # 모든 단어를 사용하지만 인덱스 1에는 단어 'OOV'를 할당한다.
src_tokenizer.fit_on_texts( sentences )
tar_tokenizer = Tokenizer( lower = False ) # 태깅 정보들은 내부적으로 대문자를 유지한채로 저장
tar_tokenizer.fit_on_texts( ner_tags )

In [ ]:
vocab_size = len( src_tokenizer.word_index ) + 1
tag_size = len( tar_tokenizer.word_index ) + 1
print( '단어 집합의 크기 : {}'.format( vocab_size ) )
print( '개체명 태깅 정보 집합의 크기 : {}'.format( tag_size ) )

In [ ]:
print( '단어 OOV의 인덱스 : {}'.format( src_tokenizer.word_index[ 'OOV' ] ) )

### word representation

In [ ]:
X_train = src_tokenizer.texts_to_sequences( sentences )
y_train = tar_tokenizer.texts_to_sequences( ner_tags )

In [ ]:
print( X_train[ 0 ] )
print( y_train[ 0 ] )

In [ ]:
index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word
index_to_ner[ 0 ] = 'PAD'

In [ ]:
print( index_to_ner )

In [ ]:
decoded = []
for index in X_train[ 0 ] : # 첫번째 샘플 안의 인덱스들에 대해서
    decoded.append( index_to_word[ index ] ) # 다시 단어로 변환

print( '기존의 문장 : {}'.format( sentences[ 0 ] ) )
print( '디코딩 문장 : {}'.format( decoded ) )

In [ ]:
max_len = 70
X_train = pad_sequences( X_train, padding = 'post', maxlen = max_len )
# X_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.

y_train = pad_sequences( y_train, padding = 'post', maxlen = max_len )
# y_train의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자0으로 채움.

### 훈련/테스트 데이터

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X_train, y_train, test_size = .2, random_state = 777 )

In [ ]:
y_train = to_categorical( y_train, num_classes = tag_size )
y_test = to_categorical( y_test, num_classes = tag_size )

In [ ]:
print( '훈련 샘플 문장의 크기 : {}'.format( X_train.shape ) )
print( '훈련 샘플 레이블의 크기 : {}'.format( y_train.shape ) )
print( '테스트 샘플 문장의 크기 : {}'.format( X_test.shape ) )
print( '테스트 샘플 레이블의 크기 : {}'.format( y_test.shape ) )

### 양방향 LSTM + CRF를 이용한 개체명 인식 모델

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
model = Sequential()
model.add( Embedding( input_dim = vocab_size, output_dim = 20, input_length = max_len, mask_zero = True ) )
model.add( Bidirectional( LSTM( units = 50, return_sequences = True, recurrent_dropout = 0.1 ) ) )
model.add( TimeDistributed( Dense( 50, activation = "relu" ) ) )
crf = CRF( tag_size )
model.add( crf )

In [ ]:
model.compile( optimizer = "rmsprop", loss = crf.loss_function, metrics = [ crf.accuracy ] )
history = model.fit( X_train, y_train, batch_size = 32, epochs = 5, validation_split = 0.1, verbose = 1 )

In [ ]:
print( "\n 테스트 정확도: %.4f" % ( model.evaluate( X_test, y_test )[ 1 ] ) )

In [ ]:
i=13 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict( np.array( [ X_test[ i ] ] ) ) # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = np.argmax( y_predicted, axis = -1 ) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true = np.argmax( y_test[ i ], -1 ) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.

print( "{:15}|{:5}|{}".format( "단어", "실제값", "예측값" ) )
print( 35 * "-")

for w, t, pred in zip( X_test[ i ], true, y_predicted[ 0 ]) :
    if w != 0: # PAD값은 제외함.
        print( "{:17}: {:7} {}".format( index_to_word[ w ], index_to_ner[ t ], index_to_ner[ pred ] ) )